## Introduction to Algorithmic Collusion

## Background
The Netherlands Authority for Consumers and Markets (ACM) is an antitrust authority. Its primary goal is to ensure fair competition and protect consumers from anticompetitive practices. With the rise of artificial intelligence (AI) and machine learning, a new challenge has emerged: algorithmic collusion. This phenomenon occurs when algorithms, designed to optimize pricing strategies, inadvertently or deliberately engage in collusive behaviour, leading to supra-competitive prices. 

## Understanding Algorithmic Collusion
Algorithmic collusion refers to the situation where pricing algorithms, often used by companies to set prices dynamically, result in higher prices than in traditional competition. Unlike human collusion, where firms explicitly agree to fix prices, algorithmic collusion can occur without any direct communication between firms. Instead, it arises from the algorithms' learning processes and their responses to market conditions. 
The outcomes of theoretical simulations of early studies reported in e.g. Calvano et al. (2020) and Klein (2021) have generated algorithmic collusion. Recently, Xavier Lambin  and co-authors have challenged these results and attributed algorithmic collusion to the simultaneous setup of the simulations, see e.g. Abada and Lambin (2023) and Lambin (2024). 


# Key Concerns
1.	Simultaneous Experimentation: Algorithms often experiment with different pricing strategies simultaneously. This can lead to high prices as they learn from each other's actions.
2.	Learning Inertia: Reinforcement learning algorithms may exhibit inertia, where initial high prices persist over time due to the learning process.
3.	Memoryless Environments: Even in environments where algorithms do not consider past actions, high prices can still emerge.
Policy Implications
Traditional antitrust measures may not be sufficient to address algorithmic collusion. Reducing market transparency, for example, may not effectively lower prices. Instead, the ACM needs to consider new approaches, such as enforcing sequential learning, where algorithms take turns exploring pricing strategies.

ME case: Running Simulations
To better understand algorithmic collusion, the ACM has hired you as a consultant to run simulations. Here’s a step-by-step guide:
1.	Set Up the Environment: Use a programming language like R or Python (and libraries such as NumPy and Matplotlib).
2.	Define the Market: Set up a market with two firms, each using a Q-learning algorithm. You may consider either homogeneous or differentiated products, competition in either prices (Bertrand) or quantities (Cournot) and whether prices are set either simultaneously (Lambin, 2024) or sequentially (Klein, 2021). One combination suffices.
3.	Implement Q-Learning: Create a Q-learning algorithm that simulates decisions in your duopoly market.
4.	Run Simulations: Allow the algorithms to explore and exploit strategies over multiple iterations similar as in the literature.
5.	Analyse Results: Observe whether the algorithms converge to high prices and compare the results to the findings in the literature.
6.	Policy Recommendations: Use your simulations to report to the ACM whether algorithmic collusion is a threat (or not). It would be nice to calculate welfare effects, such as the gain in producer surplus, the loss of consumer surplus and the deadweight loss.   
By running these simulations, you gain hands-on experience with the dynamics of algorithmic collusion and explore potential policy interventions to mitigate its effects.
Summary
Algorithmic collusion presents a significant challenge for antitrust authorities. By understanding the mechanisms behind it and exploring innovative solutions, we can better protect consumers and ensure fair competition in the digital age.
Resources
The source code of Calvano et al. (2020) is available at the website of the American Economic Review. Unfortunately, their source code is in R instead of Python. Also, Abada and Lambin (2023) provide their source code in R for a duopoly with inventories (not asked). 
For the use of ChatGPT or similar AI the plagiarism rules of this course apply. 

In this assignment we follow the work of Calvano et al (2020) in order to create a simulation of the algorithmic collusion case.

Consider two firms (1 and 2) that produce differentiated products. Demand for wach product is defined by the following function: 



$$ q_{1, t} = \frac{e^\frac{2 - p_{1, t}}{1/4}}{e^{\frac{2 - p_{1, t}}{1/4}} +  e^{\frac{2 - p_{2, t}}{1/4}} + 1}$$

$$ q_{2, t} = \frac{e^\frac{2 - p_{1, t}}{1/4}}{e^{\frac{2 - p_{1, t}}{1/4}} +  e^{\frac{2 - p_{2, t}}{1/4}} + 1}$$


Firms are interested in profit maximization. They will choose prices such that they maximize their profits. In these equations we have: 


The profit each firm earns is given by: 

$$ \pi_{1, t} = (p_{1, t} - 1) q_{1, t}$$

$$ \pi_{2, t} = (p_{2, t} - 1) q_{2, t}$$

Where we recognize that 1 represent the marginal cost of the firm. 

To create our simulation we will use object-oriented programming. Please before continuing with the assignment, if this concept is new for you, watch the following Youtube video. We parametrize our simulation using the values from baseline parametrization and initalization from Calvano et al. (2020). 

In [28]:
# Import the following required modules. 
import numpy as np
from scipy.optimize import minimize

In [ ]:
class Market():

    '''
    Define the firm class considering all the important characteristics a firm 
    will have
    '''
    def __init__(self, 
        number_of_firms = 2, 
        number_of_periods = 0 ):

        self.number_of_firms = number_of_firms
        self.numer_of_periods = number_of_periods

        self.a  = 2
        self.c = 1
        self.mu = 1/4

        # Initialize each period price as a vector of zeros
        self.price = np.zeros((self.number_of_firms, self.numer_of_periods))

        # Initialize each periof quantitity as a vector of zeros
        self.quantity = np.zeros((self.number_of_firms, self.numer_of_periods))

        self.profits = np.zeros((self.number_of_firms, self.numer_of_periods))


    def firm_demand(self):

        ul  = np.exp((self.a - self.price)/(self.mu))
        sum_ul = np.sum(ul, axis = 0)
        q = ul/(sum_ul+1)

        return q
    
    def construct_Jacobian(self): 
        """ Formulas for the matrix of first order conditions of market
        shares with respect to prices 

        Args:
            v_p (float): random consumer demand shocks
            all_probs (float): the probability a consumer i buy a product j 

        Returns:
            float matrix : the Jacobian matrix of shares with respect to prices 

        updated version that should be giving the corrent result in the end 
        This needs to be further debugged because i think it doesn't some
        across correctly 
        """
        J = np.zeros((self.number_of_firms, self.number_of_firms))
        for i in range(J.shape[0]):
            p1 = all_probs[i, :]
            for j in range(J.shape[1]):
                if i == j:
                    J[i, j] = np.sum(alphas * p1 - alphas * (p1 ** 2))/self.n_consumers
                else: 
                    p2 = all_probs[j, :]
                    J[i, j] = np.sum(-alphas * p1 * p2)/self.n_consumers
        return J

    
    def firm_profit(self, prices):

        self.price = prices
        self.quantity = self.firm_demand()

        pi = (self.price - self.c)*self.quantity

        return - pi


    def compute_NB_equilibrium(self): 

        price_initial = np.ones(self.number_of_firms)

        res = minimize(self.firm_profit, price_initial)

        return res



    def compute_monopoly_equilibiurm(self): 

        return 

    def compute_Q_learning_equilibirum(self):

        return 


Exercise 1: 

Compute the Bertrand-Nash equilibiurm


In [39]:
# Your code here for computing Nash Bertrand Equilibiurm 

market = Market()

vb = market.compute_NB_equilibrium()

print(vb)

ValueError: The user-provided objective function must return a scalar value.

## A. 

Exercise 2: 

Compute the Monopoly (perfect collusion equilibiurm)

In [1]:
# Your code here for computing Nash Bertrand Equilibiurm 

Exercise 3: 

Compute the Q-learning algorith: 

References

Abada, Ibrahim and Xavier Lambin. 2023. Artificial Intelligence: Can Seemingly Collusive Outcomes Be Avoided? Management Science 69 (9): 5042-5065. DOI: 10.1287/mnsc.2022.4623

Calvano, Emilio, Giacomo Calzolari, Vincenzo Denicolò, and Sergio Pastorello. 2020. "Artificial Intelligence, Algorithmic Pricing, and Collusion." American Economic Review 110 (10): 3267–97. DOI: 10.1257/aer.20190623

Klein, Timo. 2021. “Autonomous algorithmic collusion: Q-learning under sequential pricing.” RAND Journal of Economics 52 (3): 538-558. DOI: 10.1111/1756-2171.12383

Lambin, Xavier. 2024. Less than meets the eye: simultaneous experiments as a source of algorithmic seeming collusion. Available at SSRN: https://ssrn.com/abstract=4498926 or http://dx.doi.org/10.2139/ssrn.4498926


Assignment developed by Harold Houba and Ana Popovici